# Fancy Indexing

## Exploring Fancy Indexing

Fancy indexing is conceptually simple: **it means passing an array of indices to access multiple array elements at once**.


In [2]:
import numpy as np
rand = np.random.RandomState(42)

x = rand.randint(100, size=10)
print(x)

[51 92 14 71 60 20 82 86 74 74]


Suppose we want to access three different elements. We could do it like this:

In [ ]:
[x[3], x[7], x[2]]

[71, 86, 14]

Alternatively, we can pass a single list or array of indices to obtain the same result:

In [ ]:
ind = [3, 7, 2]
x[ind]

array([71, 86, 14])

When **using fancy indexing**, **the shape of the result reflects the shape of the index arrays** rather than the shape of the *array being indexed*:

In [3]:
ind = np.array([[3, 7],
                [4, 5]])
print(ind)
x[ind]

[[3 7]
 [4 5]]


array([[71, 86],
       [60, 20]])

**Fancy indexing also works in multiple dimensions**. Consider the following array:

In [6]:
X = np.arange(12).reshape((3, 4))
X

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

Like with standard indexing, the first index refers to the row, and the second to the column:

In [7]:
row = np.array([0, 1, 2])
col = np.array([2, 1, 3])
print("row: ", row)
print("col: ", col)
X[row, col]

row:  [0 1 2]
col:  [2 1 3]


array([ 2,  5, 11])

Notice that the first value in the result is ``X[0, 2]``, the second is ``X[1, 1]``, and the third is ``X[2, 3]``.

The pairing of indices in **fancy indexing follows all the broadcasting rules**

So, for example, if we **combine a column vector and a row vector within the indices**, we get a **two-dimensional result**:

In [ ]:
X[row[:, np.newaxis], col]

array([[ 2,  1,  3],
       [ 6,  5,  7],
       [10,  9, 11]])

In [ ]:
row[:,]

array([0, 1, 2])

In [ ]:
row[:, np.newaxis]

array([[0],
       [1],
       [2]])

Here, each row value is matched with each column vector, exactly as we saw in broadcasting of arithmetic operations.
For example:

In [ ]:
row[:, np.newaxis] * col

array([[0, 0, 0],
       [2, 1, 3],
       [4, 2, 6]])

It is always important to **remember with fancy indexing that the return value reflects the broadcasted shape of the indices**, rather than the shape of the array being indexed.

## Combined Indexing

For even more powerful operations, fancy indexing can be combined with the other indexing schemes we've seen:

In [ ]:
print(X)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


We can **combine fancy and simple indices**:

In [ ]:
X[2, [2, 0, 1]]

array([10,  8,  9])

We can also combine **fancy indexing with slicing**:

In [ ]:
X[1:, [2, 0, 1]]

array([[ 6,  4,  5],
       [10,  8,  9]])

And we can combine fancy indexing with masking:

In [11]:
mask = np.array([1, 0, 1, 0], dtype=bool)
print(mask)
print("row", row)
print(X)
print(row[:, np.newaxis])
X[row[:, np.newaxis], mask]

[ True False  True False]
row [0 1 2]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[0]
 [1]
 [2]]


array([[ 0,  2],
       [ 4,  6],
       [ 8, 10]])

All of these indexing options combined lead to a very flexible set of operations for accessing and modifying array values.

In [ ]:

row[:, np.newaxis]

## Modifying Values with Fancy Indexing

Just as fancy indexing can be used to access parts of an array, it can also be used to modify parts of an array.
For example, imagine we have an array of indices and we'd like to set the corresponding items in an array to some value:

In [ ]:
x = np.arange(10)
print(x)
i = np.array([2, 1, 8, 4])
x[i] = 99
print(x)

[0 1 2 3 4 5 6 7 8 9]
[ 0 99 99  3 99  5  6  7 99  9]


We can use any assignment-type operator for this. For example:

In [ ]:
x[i] -= 10
print(x)

[ 0 89 89  3 89  5  6  7 89  9]


Notice, though, that repeated indices with these operations can cause some potentially unexpected results. Consider the following:

#Corner Cases - Ignore

In [ ]:
x = np.zeros(10)
x[[0, 0]] = [4, 6]
print(x)

Where did the 4 go? The result of this operation is to first assign ``x[0] = 4``, followed by ``x[0] = 6``.
The result, of course, is that ``x[0]`` contains the value 6.

Fair enough, but consider this operation:

In [ ]:
i = [2, 3, 3, 4, 4, 4]
x[i] += 1
x

In [ ]:
i = [2, 3, 3, 4, 4, 4]
x[i]

You might expect that ``x[3]`` would contain the value 2, and ``x[4]`` would contain the value 3, as this is how many times each index is repeated. Why is this not the case?
Conceptually, this is because ``x[i] += 1`` is meant as a shorthand of ``x[i] = x[i] + 1``. ``x[i] + 1`` is evaluated, and then the result is assigned to the indices in x.
With this in mind, it is not the augmentation that happens multiple times, but the assignment, which leads to the rather nonintuitive results.

So what if you want the other behavior where the operation is repeated? For this, you can use the ``at()`` method of ufuncs (available since NumPy 1.8), and do the following:

In [ ]:
x = np.zeros(10)
np.add.at(x, i, 1)
print(x)

The ``at()`` method does an in-place application of the given operator at the specified indices (here, ``i``) with the specified value (here, 1).
Another method that is similar in spirit is the ``reduceat()`` method of ufuncs, which you can read about in the NumPy documentation.